In [3]:
import pyodbc

# 연결 설정
server = '210.180.103.100'  # SQL Server의 주소 또는 이름
database = 'QMSKPI'  # 연결할 데이터베이스 이름
username = 'hns_prm'  # SQL Server 사용자명
password = '@tjqltmrhksflxla1'  # 사용자의 비밀번호

# 연결 문자열 생성
connection_string = f'DRIVER=ODBC Driver 13 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'

# 데이터베이스 연결
try:
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()

    # SQL 쿼리 실행
    sql_query = """
    SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED 

SET ARITHABORT OFF 
SET ARITHIGNORE OFF
SET ANSI_WARNINGS OFF 

declare @S_DATE nvarchar(10)    
declare @E_DATE nvarchar(10)    
declare @변경종류구분 nvarchar(20)    

SET @S_DATE=dateadd(month,-1,GETDATE())  -- 완료통보일 시작    
SET @E_DATE= GETDATE()  				 -- 완료통보일 끝      

SELECT	YY ,MM ,지역 ,지점명 ,주차 ,WK
		 --,SUM(W_DAY) [W_DAY]
		--,CASE WHEN DATEPART(WEEKDAY,기준일) = 7 THEN 0.5 WHEN DATEPART(WEEKDAY,기준일) = 1 THEN 0 ELSE 1 END [W_DAY]
		,SVSET_ORD_NUM 오더번호 ,SVC_MGMT_NUM 서비스관리번호 
		,FST_SCHDLR_ID 최초할당ID
		,FST_DEPT_NM 최초할당자
		,LAST_SCHDLR_ID 최종할당ID
		,LST_DEPT_NM 최종할당자 ,LST_OPERTR_ID 작업자ID ,LST_OPERTR_ID_NM 작업자명 ,OPERTR_ID
		,SUM(CASE WHEN 주택개통건 = 'Y' THEN 1 ELSE 0 END) 승주대상
		,SUM(CASE WHEN 주택개통건 = 'Y' THEN 1 ELSE 0 END) 승주비대상
		,COUNT(*) 합계
		,SUM(CASE WHEN 구성원승주 = 'Y' AND 고소차승주 <> '출동/수행' THEN 1 ELSE 0 END) 구성원승주
		,SUM(CASE WHEN 고소차할당 = '할당' THEN 1 ELSE 0 END) 고소차할당
		,SUM(CASE WHEN 고소차할당 = '할당' AND 고소차승주 = '출동/수행' THEN 1 ELSE 0 END) 고소차승주
		,SUM(CASE WHEN 공사건 = 'Y' THEN 1 ELSE 0 END) 공사건
		,SUM(CASE WHEN 구성원승주 = 'Y' AND 고소차승주 = '출동/수행' THEN 1 ELSE 0 END) [승주중복]
		,SUM(CASE WHEN 구성원승주 = 'Y' THEN 1 ELSE 0 END) 구성원승주합계
		FROM(

			SELECT		CAST(A.STAT_DT AS DATE)기준일 ,DATEPART(YY,A.STAT_DT)YY ,DATEPART(MM,A.STAT_DT)MM ,DATEPART(DD,A.STAT_DT)DD
						,case	when datepart(weekday, STAT_DT) =1 then convert(varchar(10),datepart(week,STAT_DT)-1) else convert(varchar(10),datepart(week,STAT_DT)) end	[WK]
						,(case	when datepart(weekday, STAT_DT) =1 then convert(varchar(10),datepart(week,STAT_DT)-1) else convert(varchar(10),datepart(week,STAT_DT)) end)
						-(CASE WHEN DATEPART(weekDAY,DATEADD(DD,-(DAY(STAT_DT)-1),STAT_DT)) = 1 then DATEPART(wk,DATEADD(DD,-(DAY(STAT_DT)-1),STAT_DT)) -1 ELSE DATEPART(wk,DATEADD(DD,-(DAY(STAT_DT)-1),STAT_DT)) END)+1 [주차]
						--,CASE WHEN DATEPART(WEEKDAY,STAT_DT) = 7 THEN 0.5 WHEN DATEPART(WEEKDAY,STAT_DT) = 1 THEN 0 ELSE 1 END [W_DAY]
						,B.NWHQ 지역 ,B.MG_CO_NM_SUM 지점명 --,RIGHT(OPER_GRP_NM,4) 작업그룹
						,SVSET_ORD_NUM ,SVC_MGMT_NUM ,REQ_RCV_TYP_NM ,SVC_CHG_CD ,SVC_CHG_NM ,SVC_CHG_RSN_CD ,SVC_CHG_RSN_NM ,SVC_TECH_MTHD_CD ,SVC_TECH_MTHD_NM ,BLD_CL_CD ,BLD_CL_NM ,SVC_NM
						,SVSET_FNSH_NOTI_DTM ,FST_SCHDLR_ID ,FST_DEPT_NM ,LAST_SCHDLR_ID ,LST_DEPT_NM ,LST_OPERTR_ID ,LST_OPERTR_ID_NM ,OPERTR_ID
						,CASE WHEN FIRST_POLCLIM_OBJ_YN = 'Y' OR (SVC_CHG_CD = '기술방식변경' AND POLQUO_ST_NM = '할당' ) THEN 'Y' ELSE 'N' END 주택개통건
						,FIRST_POLCLIM_OBJ_YN [주택대상] ,POLCLIM_OBJ_YN ,POLCLIM_OPER_YN [구성원승주] ,FCLT_SET_LOC_CD ,POLQUO_ST_NM [고소차할당],POLOPER_REQ_NM ,POLOPER_ST_NM [고소차승주],POLOPER_NTRT_RSN_NM ,CNSTRCT_REQ_NUM ,CNSTRCT_REQ_DTM ,CNSTRCT_FNSH_DTM ,CNSTRCT_REQ_RSN_NM ,CNSTRCT_FNSH_YN [공사건] 


			FROM		ISSUE_SB_DBM A
						LEFT OUTER JOIN ( SELECT DISTINCT MSVC_ORG_ID,WRK_CO_ID_SUM,MG_CO_NM_SUM,NWHQ,TEAM,TYPE,HNS_YN FROM Teammapping_new_MAP WHERE ISNEW = 'Y') B ON A.MSVCORG_ID = B.MSVC_ORG_ID	

			WHERE		A.STAT_DT >= @S_DATE AND A.STAT_DT < @E_DATE
						AND SVC_CHG_CD in ('A1', 'H1', 'C8')
						AND	A.SVC_TECH_MTHD_CD IN ('B0032','B0036','B0039','B0045','B0048','B0049','TB001','TB002','TB003','TB005','TB006','TB007','TB009')
						AND MASS_CO_CL_CD = '1'
						AND SIMPL_ADDR_CHG_YN = 'N'
						AND BLD_CL_CD <> 'A2'
						AND SUBSTRING(UNIT_OPER_CD, 4,1) <> 3
						AND TYPE IN ('Home센터')--,'기술센터')

						--and b.MG_CO_NM_SUM like '%강남%'
						--and b.NWHQ like '%서부'

			GROUP BY	CAST(A.STAT_DT AS DATE) ,DATEPART(YY,A.STAT_DT) ,DATEPART(MM,A.STAT_DT) ,DATEPART(DD,A.STAT_DT)
						,case	when datepart(weekday, STAT_DT) =1 then convert(varchar(10),datepart(week,STAT_DT)-1) else convert(varchar(10),datepart(week,STAT_DT)) end
						,(case	when datepart(weekday, STAT_DT) =1 then convert(varchar(10),datepart(week,STAT_DT)-1) else convert(varchar(10),datepart(week,STAT_DT)) end)
						-(CASE WHEN DATEPART(weekDAY,DATEADD(DD,-(DAY(STAT_DT)-1),STAT_DT)) = 1 then DATEPART(wk,DATEADD(DD,-(DAY(STAT_DT)-1),STAT_DT)) -1 ELSE DATEPART(wk,DATEADD(DD,-(DAY(STAT_DT)-1),STAT_DT)) END)+1
						,CASE WHEN DATEPART(WEEKDAY,STAT_DT) = 7 THEN 0.5 WHEN DATEPART(WEEKDAY,STAT_DT) = 1 THEN 0 ELSE 1 END
						,B.NWHQ ,B.MG_CO_NM_SUM,RIGHT(OPER_GRP_NM,4)
						,SVSET_ORD_NUM ,SVC_MGMT_NUM ,REQ_RCV_TYP_NM ,SVC_CHG_CD ,SVC_CHG_NM ,SVC_CHG_RSN_CD ,SVC_CHG_RSN_NM ,SVC_TECH_MTHD_CD ,SVC_TECH_MTHD_NM ,BLD_CL_CD ,BLD_CL_NM ,SVC_NM
						,SVSET_FNSH_NOTI_DTM ,FST_SCHDLR_ID ,FST_DEPT_NM ,LAST_SCHDLR_ID ,LST_DEPT_NM ,LST_OPERTR_ID ,LST_OPERTR_ID_NM ,OPERTR_ID
						,CASE WHEN FIRST_POLCLIM_OBJ_YN = 'Y' OR (SVC_CHG_CD = '기술방식변경' AND POLQUO_ST_NM = '할당' ) THEN '승주대상' ELSE '승주비대상' END
						,POLCLIM_OPER_YN ,POLCLIM_OBJ_YN ,FCLT_SET_LOC_CD ,POLQUO_ST_NM ,POLOPER_REQ_NM ,POLOPER_ST_NM ,POLOPER_NTRT_RSN_NM ,CNSTRCT_REQ_NUM ,CNSTRCT_REQ_DTM ,CNSTRCT_FNSH_DTM ,CNSTRCT_REQ_RSN_NM ,CNSTRCT_FNSH_YN ,FIRST_POLCLIM_OBJ_YN


		--	ORDER BY CAST(A.STAT_DT AS DATE)

	)Tot

GROUP BY  YY ,MM ,지역 ,지점명 ,주차 ,WK
		,SVSET_ORD_NUM  ,SVC_MGMT_NUM  ,FST_SCHDLR_ID ,FST_DEPT_NM ,LAST_SCHDLR_ID ,LST_DEPT_NM ,LST_OPERTR_ID ,LST_OPERTR_ID_NM ,OPERTR_ID ,LST_OPERTR_ID ,LST_OPERTR_ID_NM ,OPERTR_ID

ORDER BY YY ,MM ,지점명 ,주차
    """

    cursor.execute(sql_query)

    # 결과 가져오기
    rows = cursor.fetchall()

    # 컬럼 이름 출력
    columns = [column[0] for column in cursor.description]
    print("컬럼 이름:", columns)

    # 결과를 CSV 형식의 문자열로 변환
    csv_data = ','.join(columns) + '\n'
    for row in rows:
        csv_data += ','.join(map(str, row)) + '\n'

    # CSV 파일로 저장
    file_name = "mydata.csv"
    with open(file_name, mode='w', newline='') as file:
        file.write(csv_data)

    print(f"{file_name} 파일이 생성되었습니다.")

    # 연결 종료
    cursor.close()
    connection.close()

except Exception as e:
    print(f"Error: {str(e)}")








컬럼 이름: ['YY', 'MM', '지역', '지점명', '주차', 'WK', '오더번호', '서비스관리번호', '최초할당ID', '최초할당자', '최종할당ID', '최종할당자', '작업자ID', '작업자명', 'OPERTR_ID', '승주대상', '승주비대상', '합계', '구성원승주', '고소차할당', '고소차승주', '공사건', '승주중복', '구성원승주합계']
mydata.csv 파일이 생성되었습니다.
